In [4]:
import datetime
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [5]:
class Config:
    bos_token = '<s>'
    eos_token = '</s>'
    usr_token = '<usr>'
    pad_token = '<pad>'
    sys_token = '<sys>'
    unk_token = '<unk>'
    mask_token = '<mask>'
    max_length = 256
    max_turns = 8
    epochs = 2
    batch_size = 4
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    learning_rate = 1e-4
    model_name = "skt/kogpt2-base-v2"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(Config.model_name,
            bos_token=Config.bos_token, eos_token=Config.eos_token, unk_token=Config.unk_token,
            pad_token=Config.pad_token, mask_token=Config.mask_token, model_max_length=Config.max_length)
model = AutoModelForCausalLM.from_pretrained(Config.model_name).to(Config.device)


Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

In [7]:
model_path = 'models/model2.pth'

model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [10]:
history_limit = ['<s>']

while True:
    user_message = input("user > ")

    if user_message == "끝": break
        
    if user_message == "초기화":
        history_limit = ['<s>']
        continue
        
    if len(history_limit) == Config.max_turns * 2 + 1:
        history_limit = history_limit[: 1] + history_limit[3: ]
        
    user_message_pull = Config.usr_token + user_message + Config.sys_token

    history_limit.append(user_message_pull)

    message_ids = tokenizer.encode(''.join(history_limit), return_tensors="pt").to(Config.device)

    with torch.no_grad():
        reply_ids = model.generate(
                    message_ids,
                    max_length=256,
                    do_samples=True,
                    top_k=9,
                    top_p=0.95,
                    bad_words_ids=[[tokenizer.convert_tokens_to_ids(token)] for token in [Config.sys_token, Config.usr_token]],
                    max_new_tokens=30,
                    eos_token_id=tokenizer.eos_token_id
        )
    decoded_ids = reply_ids[0, message_ids.shape[-1]: ]
    if decoded_ids[-1] ==tokenizer.eos_token_id:
        decoded_ids = decoded_ids[: -1]
            
    decoded_message = tokenizer.decode(decoded_ids)
    
    history_limit.append(decoded_message)

    print(history_limit)
    

user > 안녕
['<s>', '<usr>안녕<sys>', '안녕하세요? 형님! 형님! 형님! 형님! 저는 지금 형님과 함께 살고 있어요! ᄏ']
user > 나랑?
['<s>', '<usr>안녕<sys>', '안녕하세요? 형님! 형님! 형님! 형님! 저는 지금 형님과 함께 살고 있어요! ᄏ', '<usr>나랑?<sys>', '형님! 형님! ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형']
user > 앙
['<s>', '<usr>안녕<sys>', '안녕하세요? 형님! 형님! 형님! 형님! 저는 지금 형님과 함께 살고 있어요! ᄏ', '<usr>나랑?<sys>', '형님! 형님! ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형', '<usr>앙<sys>', '형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ']
user > 앙
['<s>', '<usr>안녕<sys>', '안녕하세요? 형님! 형님! 형님! 형님! 저는 지금 형님과 함께 살고 있어요! ᄏ', '<usr>나랑?<sys>', '형님! 형님! ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형', '<usr>앙<sys>', '형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ', '<usr>앙<sys>', '형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ']
user > 앙
['<s>', '<usr>안녕<sys>', '안녕하세요? 형님! 형님! 형님! 형님! 저는 지금 형님과 함께 살고 있어요! ᄏ', '<usr>나랑?<sys>', '형님! 형님! ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형', '<usr>앙<sys>', '형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ', '<usr>앙<sys>', '형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ', '<usr>앙<sys>', '형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ 형님 ᄏᄏ']
user > 앙
['<s>', '<usr>안녕<sys>', '안

KeyboardInterrupt: Interrupted by user